In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn import metrics
import tensorflow as tf
import tensorflow.contrib.slim as slim
from networkx import to_numpy_matrix, karate_club_graph
import networkx as nx

In [2]:
zkc = karate_club_graph()
order = sorted(list(zkc.nodes()))
A = to_numpy_matrix(zkc, nodelist=order)
I = np.eye(zkc.number_of_nodes())          
A_hat = A + I
D_hat = np.array(np.sum(A_hat, axis=0))[0] # 度矩阵
D_hat_ = np.array(np.diag(D_hat))

In [3]:
print(A_hat, A_hat.shape, len(A_hat), A_hat[0])
print(D_hat, D_hat.shape)

[[1. 1. 1. ... 1. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 1. 0.]
 ...
 [1. 0. 0. ... 1. 1. 1.]
 [0. 0. 1. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]] (34, 34) 34 [[1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]]
[17. 10. 11.  7.  4.  5.  5.  5.  6.  3.  4.  2.  3.  6.  3.  3.  3.  3.
  3.  4.  3.  3.  3.  6.  4.  4.  3.  5.  4.  5.  5.  7. 13. 18.] (34,)


In [4]:
sess = tf.Session()

dic = {}
dic['Officer'] = 0
dic['Mr. Hi'] = 1
labels = np.zeros((len(zkc), 2))
for i in range(len(zkc)):
    if dic[zkc.nodes[i]['club']] == 0:
        labels[i][0] = 1
        labels[i][1] = 0
    else:
        labels[i][0] = 0
        labels[i][1] = 1

## gcn

In [5]:
def gcn_layer(A_hat, D_hat, inputs):
    tmp = D_hat**-1 * A_hat * inputs
    out = slim.fully_connected(tmp, 32, activation_fn=tf.nn.relu)
    out = slim.fully_connected(out, 2, activation_fn=tf.nn.softmax)
    return out

inputs = tf.placeholder(shape=[None, 34], dtype=tf.float32)
outputs_gcn = gcn_layer(A_hat, D_hat, inputs)

In [30]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=outputs_gcn, labels=labels))
opt = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

sess.run(tf.global_variables_initializer())

In [31]:
for i in range(1000):
    error, _ = sess.run([loss, opt], feed_dict={inputs:I})
    if i % 200 == 0:
        print('epoch', i, error)
    
print('last error', error)

epoch 0 0.69573176
epoch 200 0.66557425
epoch 400 0.63263017
epoch 600 0.59604
epoch 800 0.5571224
last error 0.51801807


In [32]:
preds = sess.run(outputs_gcn, feed_dict={inputs:I})

In [33]:
c = 0
for i in range (34):
    print('labels:', np.argmax(labels[i]), 'preds:', np.argmax(preds[i]), preds[i])
    if np.argmax(labels[i]) != np.argmax(preds[i]):
        c += 1
        print(preds[i])
print('acc:', 1 - c/34)

labels: 1 preds: 1 [0.02872175 0.97127825]
labels: 1 preds: 1 [0.27755228 0.72244775]
labels: 1 preds: 1 [0.22032213 0.77967787]
labels: 1 preds: 1 [0.3837679  0.61623216]
labels: 1 preds: 1 [0.26497868 0.73502135]
labels: 1 preds: 1 [0.12208049 0.87791955]
labels: 1 preds: 1 [0.30531394 0.69468606]
labels: 1 preds: 0 [0.53346616 0.46653387]
[0.53346616 0.46653387]
labels: 1 preds: 1 [0.22304103 0.776959  ]
labels: 0 preds: 0 [0.56227666 0.4377233 ]
labels: 1 preds: 1 [0.26393977 0.73606026]
labels: 1 preds: 1 [0.31254023 0.68745977]
labels: 1 preds: 1 [0.30716547 0.69283456]
labels: 1 preds: 1 [0.3432389  0.65676117]
labels: 0 preds: 0 [0.6562055  0.34379447]
labels: 0 preds: 0 [0.62545455 0.37454545]
labels: 1 preds: 1 [0.27953702 0.7204629 ]
labels: 1 preds: 1 [0.36003712 0.6399629 ]
labels: 0 preds: 0 [0.6042018 0.3957982]
labels: 1 preds: 1 [0.3592061 0.640794 ]
labels: 0 preds: 0 [0.6522426  0.34775737]
labels: 1 preds: 1 [0.42397293 0.5760271 ]
labels: 0 preds: 0 [0.6678991 0.33